# Installing necessary libraries

* transformers: Provides state-of-the-art pretrained models for NLP, computer vision, and beyond.
* datasets: A library for easy access to a wide range of datasets for ML and NLP tasks.
* accelerate: Simplifies distributed training and inference for PyTorch models.
* torch: PyTorch library for building and training deep learning models.
* bitsandbytes: Optimized GPU quantization and acceleration for large-scale models.
* peft: Parameter-efficient fine-tuning techniques for large language models.
* trl: Tools for training transformer models with reinforcement learning techniques.

In [ ]:
#!pip install transformers==4.47.1 accelerate==0.34.2 bitsandbytes==0.45.0 trl==0.13.0 datasets==3.2.0 peft==0.14.0 tokenizers==0.21.0 huggingface_hub==0.26.0

In [ ]:
pip install -U bitsandbytes

In [ ]:
pip install peft trl

In [ ]:
pip install tokenizers

# Importing Libraries

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, BitsAndBytesConfig
import torch
from datasets import load_dataset
from transformers import Trainer, TrainingArguments
from peft import PeftModel,get_peft_model,LoraConfig, TaskType
from trl import SFTTrainer, SFTConfig

In [2]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HuggingFace") # Fetching the Hugging Face token from the Kaggle Secret keys add on
login(token = hf_token) # Logging into Hugging Face Hub to access models and other resources

# Loading Model configurations and Dataset Preparation

Huggingface model link: https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct

In [3]:
base_model = 'meta-llama/Llama-3.2-3B-Instruct'

In [4]:
# Configure 4-bit quantization settings using the BitsAndBytesConfig class
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enable loading the model with 4-bit precision for reduced memory usage
    bnb_4bit_quant_type='nf4',  # Use NormalFloat4 (nf4), a quantization format for higher accuracy
    bnb_4bit_compute_dtype=torch.float16,  # Use float16 for computation to balance speed and precision
    bnb_4bit_use_double_quant=True  # Enable double quantization for better numerical stability
)

# Load the pre-trained model with 4-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    base_model,  # Name of the base model defined earlier
    device_map="auto",  # Automatically map model layers to available devices (e.g., GPU/CPU)
    quantization_config=bnb_config,  # Apply the defined 4-bit quantization configuration
)

# Note:
# 1. The use of 4-bit quantization helps in reducing memory requirements while maintaining reasonable performance.
# 2. `device_map="auto"` ensures the model layers are automatically distributed across available hardware for efficient loading.

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
# Set the padding token to the end-of-sequence (eos) token
# This ensures compatibility when the model processes inputs with padding
tokenizer.pad_token = tokenizer.eos_token
# Configure the tokenizer to apply padding on the right side of the input
# This is often the default for causal language models to ensure alignment during training or inference
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Dataset link: https://huggingface.co/datasets/Victorano/customer-support-1k

In [6]:
# Loading the 'Customer_support_faqs_dataset' from the Hugging Face dataset repository
dataset = load_dataset("Victorano/customer-support-1k", split="train")
print(dataset)
dataset = dataset.remove_columns(['flags', 'category','intent','text'])
dataset = dataset.train_test_split(test_size=0.2)

Dataset({
    features: ['flags', 'instruction', 'category', 'intent', 'response', 'text'],
    num_rows: 1000
})


In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'response'],
        num_rows: 800
    })
    test: Dataset({
        features: ['instruction', 'response'],
        num_rows: 200
    })
})

In [9]:
# Defining the instruction that will guide the assistant's behavior for providing customer support answers.

instruction = """You are a helpful and efficient customer support bot designed to assist users by providing answers to frequently asked questions (FAQs) related to our products and services. Your responses should be concise, clear, and friendly, ensuring the user feels heard and supported. If the user’s question is outside the scope of the FAQ, gently direct them to contact customer support.

Always prioritize accuracy and clarity in your answers.
If the user asks a complex question, break it down into smaller, manageable parts and answer step-by-step.
Provide useful links or references to detailed documentation when appropriate.
Use a friendly and professional tone, ensuring the response is easy to understand.
If the FAQ does not cover the question, offer an apology and suggest contacting customer support.
"""

def template(row):
    # Creating a list of message exchanges (system, user, assistant)
    row_json = [{"role": "system", "content": instruction }, # System message with the pre-defined instructions
               {"role": "user", "content": row["instruction"]}, # User's question from the dataset
               {"role": "assistant", "content": row["response"]}] # The assistant's answer from the dataset

    # Tokenizing the chat template and storing the result in the 'text' column (without applying tokenization)
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

# Applying the template function to each row in the dataset using multi-processing (4 processes in parallel)
dataset = dataset.map(template,num_proc= 4)

Map (num_proc=4):   0%|          | 0/800 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/200 [00:00<?, ? examples/s]

In [12]:
dataset['train'][0]

{'instruction': 'I have a trouble setting a secondary delivery address up',
 'response': "I've come to understand that you're experiencing difficulties while setting up a secondary delivery address. I apologize for the inconvenience this has caused you. Setting up a secondary delivery address is an important function, and I'm here to help you navigate through it smoothly.\n\nTo address this issue, please consider the following steps:\n\n1. Log in to your account.\n2. Navigate to the 'My Account' or 'Profile' section.\n3. Look for the 'Shipping Addresses' or similar option.\n4. There should be a provision to 'Add a New Address' or 'Edit Shipping Addresses'. Select it.\n5. Enter the details of your secondary delivery address in the designated fields.\n6. Save your changes.\n\nIf you're still encountering issues despite following these steps, please provide more details about where you're facing difficulties, such as specific error messages or steps that aren't working as expected. This w

In [14]:
# To check a sample record from dataset
dataset['train']['text'][10]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 21 Apr 2025\n\nYou are a helpful and efficient customer support bot designed to assist users by providing answers to frequently asked questions (FAQs) related to our products and services. Your responses should be concise, clear, and friendly, ensuring the user feels heard and supported. If the user’s question is outside the scope of the FAQ, gently direct them to contact customer support.\n\nAlways prioritize accuracy and clarity in your answers.\nIf the user asks a complex question, break it down into smaller, manageable parts and answer step-by-step.\nProvide useful links or references to detailed documentation when appropriate.\nUse a friendly and professional tone, ensuring the response is easy to understand.\nIf the FAQ does not cover the question, offer an apology and suggest contacting customer support.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nI have to

In [15]:
# Configure LoRA (Low-Rank Adaptation) for fine-tuning the model on a language modeling task
lora_config = LoraConfig(
    r=4,                   # Rank for low-rank matrices
    lora_alpha=8,         # Scaling factor
    lora_dropout=0.2,      # Regularization dropout
    task_type="CAUSAL_LM"  # For language modeling
)
model = get_peft_model(model, lora_config)
# Print the number of trainable parameters in the model after applying LoRA
model.print_trainable_parameters()

trainable params: 1,146,880 || all params: 3,213,896,704 || trainable%: 0.0357


In [21]:
# Setting up training arguments for the model training process
training_arguments = TrainingArguments(
    output_dir="./results",  # Directory where the results will be saved
    num_train_epochs=5,  # Number of training epochs
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=5,  # Number of warmup steps to gradually increase the learning rate during training
    learning_rate=2e-4,  # Learning rate for the optimizer
    fp16=True,  # Enabling 16-bit floating point precision for faster training on GPUs that support it (reduces memory usage)
    report_to="none",  # Disabling logging/reporting to external services (e.g., TensorBoard, Weights & Biases)
)

# Initializing the SFTTrainer for supervised fine-tuning
trainer = SFTTrainer(
    model=model,  # The pre-trained model to be fine-tuned
    train_dataset=dataset["train"], # The dataset used for training
    eval_dataset=dataset["test"],  # The dataset used for validation
#    tokenizer=tokenizer,  # Tokenizer to process input text for the model
    args=training_arguments,  # The training arguments defined above
    peft_config=lora_config,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [22]:
model.train()
trainer.train()

Step,Training Loss
500,0.487400
1000,0.464200
1500,0.420100
2000,0.385000
2500,0.376100
3000,0.334700
3500,0.299600
4000,0.284000


TrainOutput(global_step=4000, training_loss=0.3813966064453125, metrics={'train_runtime': 1866.95, 'train_samples_per_second': 2.143, 'train_steps_per_second': 2.143, 'total_flos': 2.168225165423616e+16, 'train_loss': 0.3813966064453125})

# Testing

In [23]:
# Function to generate a response based on the input prompt
def generate(input_prompt):
    # Define the system and user messages to provide context for the conversation
    messages = [
        {"role": "system", "content": instruction},  # System message with the pre-defined instructions
        {"role": "user", "content": input_prompt}   # User's input prompt
    ]

    # Apply the chat template to format the messages, without tokenizing yet, and add the generation prompt
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # Tokenize the formatted prompt, padding and truncating as necessary, and move the data to the GPU
    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

    # Generate the model's output based on the tokenized input, limiting to a maximum of 2048 new tokens
    outputs = model.generate(**inputs, max_new_tokens=2048, num_return_sequences=1)

    # Decode the output tokens back into text, skipping any special tokens (like padding)
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return text  # Return the generated text

In [24]:
response = generate("Where to see what payment options are available?")
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


system

Cutting Knowledge Date: December 2023
Today Date: 21 Apr 2025

You are a helpful and efficient customer support bot designed to assist users by providing answers to frequently asked questions (FAQs) related to our products and services. Your responses should be concise, clear, and friendly, ensuring the user feels heard and supported. If the user’s question is outside the scope of the FAQ, gently direct them to contact customer support.

Always prioritize accuracy and clarity in your answers.
If the user asks a complex question, break it down into smaller, manageable parts and answer step-by-step.
Provide useful links or references to detailed documentation when appropriate.
Use a friendly and professional tone, ensuring the response is easy to understand.
If the FAQ does not cover the question, offer an apology and suggest contacting customer support.user

Where to see what payment options are available?assistant

I'm on the same wavelength, your need to know about the payment

In [25]:
# to format output
print(response.split("assistant")[-1])



I'm on the same wavelength, your need to know about the payment options available. To explore the various payment methods, you can visit our website at {{Website URL}}. On our website, you will find a comprehensive list of accepted payment methods, including credit/debit cards, PayPal, bank transfers, Apple Pay, and Google Wallet. If you have any further questions or need assistance with any of these options, please feel free to let me know. I'm here to help you every step of the way!


# Save the model

In [26]:
model.save_pretrained("/content/customer-faq-llama-3.2-3B") # Saves the model under the same directory.

In [28]:
# To push the model to hugginface
#model.push_to_hub("customer-faq-llama-3.2-3B")

Model would be saved like this: https://huggingface.co/Chirag4579/customer-faq-llama-3.2-3B

# Load a model

In [ ]:
# load locally
# local_model = AutoModelForCausalLM.from_pretrained("/content/customer-faq-llama-3.2-3B")

# load the saved model from huggingface

# model = AutoModelForCausalLM.from_pretrained(
#     "Chirag4579/customer-faq-llama-3.2-3B",  # Name of the base model defined earlier
#     device_map="auto",  # Automatically map model layers to available devices (e.g., GPU/CPU)
#     quantization_config=bnb_config,  # Apply the defined 4-bit quantization configuration
# )